In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder



llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.1)


memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=120, memory_key="chat_history", return_messages=True)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{question}"),
    ]
)


def load_memory(input):
    print(input)
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": "I live in Seoul, South Korea. What is the capital of South Korea?"})
    memory.save_context(
        {"input": question}, {"output": result.content},
    )
    print(result)

In [16]:
invoke_chain("What is the capital of South Korea?")

invoke_chain("Which country is Seoul in?")

{'question': 'I live in Seoul, South Korea. What is the capital of South Korea?'}
content='The capital of South Korea is Seoul, where you currently reside. It is a vibrant and bustling city known for its rich history, modern architecture, and delicious cuisine.'
{'question': 'I live in Seoul, South Korea. What is the capital of South Korea?'}
content='I apologize for the oversight. As you correctly mentioned, Seoul is the capital of South Korea. It is a dynamic city with a mix of traditional and modern elements, making it a fascinating place to live.'
